In [20]:
import requests # http requests
import re # regular expressions
from bs4 import BeautifulSoup # xml parsing
import pandas as pd
import regex as re
import os

In [14]:
def check_response(url):
    response = requests.get(url)
    status_code = response.status_code
    if status_code != 200:
        raise Exception(f'Error {status_code} ({requests.status_codes._codes[status_code]})')
    else:
        html_text = response.content
        print(f'Request successful')
        return html_text

def make_soup(html_text):
    return BeautifulSoup(html_text, 'html.parser')

def get_contents(soup):
    entries = []
    keys = ['datetime', 'name', 'rating', 'title', 'review', 'event_time']
    reviews = soup.find_all('div', class_ = 'styles_reviewCardInner__EwDq2')
    
    for review in reviews:
        try:
            text = review.find('p', class_ = 'typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn').get_text()
        except AttributeError:
            continue
        
        date = (review.find('time')).attrs['datetime']
        name = review.find('span', class_ = 'typography_heading-xxs__QKBS8 typography_appearance-default__AAY17').get_text()
        rating = re.search('(\d+)(?=\s*.svg)', str(review.find('div', class_ =  'star-rating_starRating__4rrcf star-rating_medium__iN6Ty')))[0]
        title = review.find('h2', class_ = 'typography_heading-s__f7029 typography_appearance-default__AAY17').get_text()
        event = review.find('p', class_ = 'typography_body-m__xgxZ_ typography_appearance-default__AAY17').get_text()[20:]
        
        values = [date, name, rating, title, text, event]
        d = dict(zip(keys, values))
        entries.append(d)

    return entries

def make_df(df, d):
    df_new = pd.DataFrame.from_dict(d)
    return pd.concat([df, df_new], ignore_index=True, axis = 0) 


In [17]:
n = 100

df = pd.DataFrame()

for i in range(1,n):
    if len(df)>=n:
        print(f'Scrape complete! \n {i} review pages have been scanned, total reviews is {len(df)}.')
        break
    url = 'https://dk.trustpilot.com/review/puregym.dk?page={}&sort=recency'.format(i)
    html = check_response(url)
    soup = make_soup(html)
    d = get_contents(soup)
    df = make_df(df, d)


Request successful
Request successful
Request successful
Request successful
Request successful
Request successful
Scrape complete! 
 7 review pages have been scanned, total reviews is 100.


In [26]:
df

,datetime,name,rating,title,review,event_time
0,2023-10-02T12:49:51.000Z,Anni,5,Det var en god oplevelse pga… bikeinstruktøren,Det var en god oplevelse pga instruktøren. Det...,02. oktober 2023
1,2023-10-03T17:21:09.000Z,Hanne Bruun Blaabjerg,5,Positimisme,"Hyggeligt sted, god atmosfære, plads til alle,...",03. oktober 2023
2,2023-09-29T12:41:44.000Z,Consumer,5,Zumba og stram op med Louise Pilgaard…,Zumba og stram op med Louise Pilgaard er noget...,29. september 2023
3,2023-10-02T12:30:58.000Z,Klaus,3,Der har efterhånden bredt sig en…,Der har efterhånden bredt sig en generel utilf...,29. september 2023
4,2023-10-03T09:32:28.000Z,Jane Rohde Olesen,4,Stille center men dårlig udluftning,"Dejligt stille center, som jeg har brug for. M...",03. oktober 2023
...,...,...,...,...,...,...
95,2023-09-25T21:43:07.000Z,Lars Mathes,4,Overordnet et rigtig godt center med…,Overordnet et rigtig godt center med god kvali...,25. september 2023
96,2023-09-25T20:10:09.000Z,Lotte,5,Et center hvor der er plads til alle.,Et center hvor der er plads til alle.Hjælpsomm...,25. september 2023
97,2023-09-25T19:55:29.000Z,Henrik Persson,4,En forholdsvis rimelig pris,"En forholdsvis rimelig pris, for maskiner der ...",25. september 2023
98,2023-09-25T18:57:02.000Z,Anna Zejden,5,Anbefales latin holdtræning til alle,Anbefales latin holdtræning til alle. Du får m...,25. september 2023


In [25]:
cwd = os.getcwd()
path = cwd + "/TrustPilot_reviews.csv"
df.to_csv(path, encoding='utf-16')

In [37]:
#https://translate.google.com/?sl=da&tl=en&text=hej.%2C!%3F%20&op=translate
table = {' ':'%20', ',':'%2C','?':'%3F'}
test = d[0]['review']
print(test)
mapping = test.maketrans(table)
print(test.translate(mapping))

Højt til loftet, og fantastisk personale som holder rent, og hele tiden er synlige.
Højt%20til%20loftet%2C%20og%20fantastisk%20personale%20som%20holder%20rent%2C%20og%20hele%20tiden%20er%20synlige.


In [62]:
def translate(str):
    url = 'https://translate.google.com/?sl=da&tl=en&text={}&op=translate'.format(str)
    print(url)
    html = check_response(url)
    soup = make_soup(html)
    f = open('recipe', 'w')
    f.write(soup.prettify())
    #text = soup.find('div', class_ = 'lRu31')

    

    #text = f.find('span', class_ = 'HwtZe')
    f.close
    
    return 

test_string = d[0]['review']
mapping = test_string.maketrans(table)

test_text = (translate(test_string.translate(mapping)))


https://translate.google.com/?sl=da&tl=en&text=Højt%20til%20loftet%2C%20og%20fantastisk%20personale%20som%20holder%20rent%2C%20og%20hele%20tiden%20er%20synlige.&op=translate
Request successful


In [55]:
test_text

[]